<a href="https://colab.research.google.com/github/fralfaro/MAT306/blob/main/docs/labs/lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT306 - Laboratorio N°03





**Objetivo**: Aplicar técnicas avanzadas de manipulación y análisis de datos con pandas sobre un conjunto real de datos de contenido de Netflix, reforzando buenas prácticas y métodos eficientes sin recurrir a `groupby`, `merge`, `pivot`, ni `join`.



**Dataset**:

Trabajaremos con el archivo `netflix_titles.csv`, que contiene información sobre los títulos disponibles en la plataforma Netflix hasta el año 2021.

| Variable       | Clase     | Descripción                                                                 |
|----------------|-----------|------------------------------------------------------------------------------|
| show_id        | caracter  | Identificador único del título en el catálogo de Netflix.                   |
| type           | caracter  | Tipo de contenido: 'Movie' o 'TV Show'.                                     |
| title          | caracter  | Título del contenido.                                                       |
| director       | caracter  | Nombre del director (puede ser nulo).                                       |
| cast           | caracter  | Lista de actores principales (puede ser nulo).                              |
| country        | caracter  | País o países donde se produjo el contenido.                                |
| date_added     | fecha     | Fecha en la que el título fue agregado al catálogo de Netflix.              |
| release_year   | entero    | Año de lanzamiento original del título.                                     |
| rating         | caracter  | Clasificación por edad (por ejemplo: 'PG-13', 'TV-MA').                      |
| duration       | caracter  | Duración del contenido (minutos o número de temporadas para series).        |
| listed_in      | caracter  | Categorías o géneros en los que está clasificado el contenido.              |
| description    | caracter  | Breve sinopsis del contenido.                                               |




In [24]:
import pandas as pd

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...



### Parte 1: Limpieza y preparación

1. Revisar y describir el dataset:

   * ¿Cuántas filas y columnas tiene?
   * ¿Qué tipos de datos hay?
   * ¿Cuántos valores nulos hay por columna?

2. Transformar la columna `date_added` a tipo fecha.

3. Crear columnas auxiliares con `assign`:

   * Año (`year_added`)
   * Mes (`month_added`)



In [28]:
# 1. ¿Cuántas filas y columnas tiene?
filas, columnas = df.shape
print(f"Filas: {filas}, Columnas: {columnas}")

#¿Qué tipos de datos hay?
print(df.dtypes)

#¿Cuántos valores nulos hay por columna?
nulos = df.isna().sum()
print(nulos)

#2. Transformar la columna `date_added` a tipo fecha.
df['date_added'] = pd.to_datetime(df['date_added'], format="%B %d, %Y", errors='coerce')
df.head()

#3. Crear columnas auxiliares con assign: Año (year_added) y Mes (month_added).
df = df.assign(
    year_added = df['date_added'].dt.year,
    month_added = df['date_added'].dt.month
)

df[['date_added', 'year_added', 'month_added']].head(5)

Filas: 8807, Columnas: 12
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


,date_added,year_added,month_added
0,2021-09-25,2021.0,9.0
1,2021-09-24,2021.0,9.0
2,2021-09-24,2021.0,9.0
3,2021-09-24,2021.0,9.0
4,2021-09-24,2021.0,9.0


## Parte 2: Técnicas avanzadas de pandas

4. Utilizar `.loc` para seleccionar películas (`type == 'Movie'`) que fueron agregadas después del año 2018.

5. Utilizar `str.contains()` y `str.extract()`:

   * Filtrar títulos que contienen la palabra 'love' (sin distinguir mayúsculas/minúsculas).
   * Extraer la duración en minutos para las películas desde la columna `duration`.

6. Aplicar `explode()` sobre la columna `listed_in` para obtener una fila por cada género.

7. Obtener un top 10 de géneros más frecuentes utilizando `value_counts()`.

8. Aplicar `where()` y `mask()` para marcar las películas de más de 120 minutos como contenido largo en una nueva columna.

9. Utilizar `.loc` para filtrar películas que cumplen con:

   * Más de 100 minutos de duración.
   * Rating igual a `'R'`.
   * País igual a `'United States'`.

10. Utilizar `.style` para formatear visualmente el top 10 de películas más largas.

In [37]:
# 4. Utilizar .loc para seleccionar películas (type == 'Movie') que fueron agregadas después del año 2018.
peliculas_post2018 = df.loc[
    (df['type'] == 'Movie') & (df['year_added'] > 2018)
]
peliculas_post2018.head(5)

# 5. Utilizar str.contains() y str.extract():
#5.1 Filtrar títulos que contienen la palabra 'love' (sin distinguir mayúsculas/minúsculas).
peliculas_love = df.loc[
    df['title'].str.contains('love', case=False, na=False)
]
peliculas_love[['title', 'type', 'year_added']].head(10)

#5.2 Extraer la duración en minutos para las películas desde la columna duration.
peliculas = df.loc[df['type'] == 'Movie']

peliculas = peliculas.assign(
    duration_min = peliculas['duration'].str.extract('(\d+)').astype(float)
)

peliculas[['title', 'duration', 'duration_min']].head(10)

#6. Aplicar explode() sobre la columna listed_in para obtener una fila por cada género.
generos_exploded = peliculas.assign(
    genre = peliculas['listed_in'].str.split(', ')
).explode('genre')

generos_exploded[['title', 'genre']].head(10)

#7. Obtener un top 10 de géneros más frecuentes utilizando value_counts().
generos_exploded['genre'].value_counts().head(10)

#8. Aplicar where() y mask() para marcar las películas de más de 120 minutos como contenido largo en una nueva columna.

peliculas = peliculas.assign(
    long_movie = peliculas['duration_min'].where(
        peliculas['duration_min'] > 120, other='No'
    )
)

# Mostrar algunas filas
peliculas[['title', 'duration_min', 'long_movie']].head(10)


#9. Utilizar .loc para filtrar películas que cumplen con: Más de 100 minutos de duración, Rating igual a 'R' y  País igual a 'United States'.
peliculas_filtradas = peliculas.loc[
    (peliculas['duration_min'] > 100) &
    (peliculas['rating'] == 'R') &
    (peliculas['country'] == 'United States')
]

# Mostrar algunas columnas relevantes
peliculas_filtradas[['title', 'duration_min', 'rating', 'country']].head(10)


#10. Utilizar .style para formatear visualmente el top 10 de películas más largas.
top_largas = peliculas.sort_values(by='duration_min', ascending=False).head(10)

top_largas[['title', 'duration_min', 'rating', 'country']].style.background_gradient(
    subset=['duration_min'], cmap='Purples'
)





<>:18: SyntaxWarning: invalid escape sequence '\d'
<>:18: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-1153609565.py:18: SyntaxWarning: invalid escape sequence '\d'
  duration_min = peliculas['duration'].str.extract('(\d+)').astype(float)


,title,duration_min,rating,country
4253,Black Mirror: Bandersnatch,312.000000,TV-MA,United States
717,Headspace: Unwind Your Mind,273.000000,TV-G,nan
2491,The School of Mischief,253.000000,TV-14,Egypt
2487,No Longer kids,237.000000,TV-14,Egypt
2484,Lock Your Girls In,233.000000,TV-PG,nan
2488,Raya and Sakina,230.000000,TV-14,nan
166,Once Upon a Time in America,229.000000,R,"Italy, United States"
7932,Sangam,228.000000,TV-14,India
1019,Lagaan,224.000000,PG,"India, United Kingdom"
4573,Jodhaa Akbar,214.000000,TV-14,India




### Pregunta Desafío

11. ¿Cuáles son las combinaciones más frecuentes de género y rating en el dataset?
    (Sugerencia: utilizar `value_counts` con `subset=["genre", "rating"]` después de aplicar `explode()`).



### Bonus: Análisis de duplicados y limpieza

12. ¿Existen películas con el mismo nombre (`title`) pero con distinto año de lanzamiento (`release_year`)?
13. ¿Cuántos títulos únicos hay en total en la columna `title`?





In [40]:
#11. ¿Cuáles son las combinaciones más frecuentes de género y rating en el dataset?
#(Sugerencia: utilizar value_counts con subset=["genre", "rating"] después de aplicar explode()).

combinaciones_top = generos_exploded.value_counts(subset=['genre', 'rating']).head(10)
combinaciones_top

#12. ¿Existen películas con el mismo nombre (title) pero con distinto año de lanzamiento (release_year)?
duplicados = peliculas.groupby('title')['release_year'].nunique()
titulos_repetidos = duplicados[duplicados > 1]

titulos_repetidos.head(10)

#NO HAY PELÍCULAS.

#13. ¿Cuántos títulos únicos hay en total en la columna title?
títulos_únicos = df['title'].nunique()
títulos_únicos



8807